In [ ]:
#this is example of interactive python server for manupulate fceux emulator
#first, load module for manipulation and wait connection for emulator

from python_server import *
waitForConnection()

In [ ]:
#Now run emulator, open ROM Castlevania (U) (PRG0) [!].nes and run lua script: fceux_listener.lua
#If emulator connects with our server without error, then all is ok. 

#Now select emulator window always visible and active (I used Dexpot https://www.dexpot.de/index.php?id=features for it)
#We can start to send commands to emulator from this notebook

In [ ]:
#We can use all standart fceux lua functions. for example:
emu.poweron()

In [ ]:
emu.framecount()

In [ ]:
emu.setrenderplanes(True, True)

In [ ]:
#Callbacks also allowed

def showFrame():
    emu.message("Hello from python. Frame:" + str(emu.framecount()))

In [ ]:
emu.registerafter(showFrame)

In [ ]:
count = 0
def onExecute(addr, size):
    global count
    count += 1
    emu.message("Exec %s count: %d"%(hex(addr),count))

In [ ]:
#Castlevania 1 example, address of IRQ on title screen
memory.registerexecute(0xC0BB, 1, onExecute)

In [ ]:
#Callbacks not very fast now (about 600 executes per frame)
memory.registerexecute(0xC037, 1, onExecute)

In [ ]:
#disable callbacks to stop freezing emulator
memory.registerexecute(0xC0BB, 1, None)
memory.registerexecute(0xC037, 1, None)

In [ ]:
#we can pause emulator to do some work with it
emu.pause()

In [ ]:
print("Emulator paused: ",emu.paused(), " Current frame:", emu.framecount())

In [ ]:
#continue running
emu.unpause()

In [ ]:
#calling emulator debugger
debugger.hitbreakpoint()

In [ ]:
emu.framecount()

In [ ]:
debugger.getinstructionscount()

In [ ]:
emu.emulating()

In [ ]:
emu.poweron()

In [ ]:
emu.print("Printing to emulator log")

In [ ]:
emu.lagged()

In [ ]:
#Read byte from RAM
hex(rom.readbyte(0x82))

In [ ]:
#Read sevaral bytes
a = memory.readbyterange(0x100, 256)
print(a)

In [ ]:
#Read screen pixel
emu.getscreenpixel(150,150,True)

In [ ]:
#conditional breakpoint - on write to memory address

memory.registerwrite(0x3F, 1, lambda addr,size: debugger.hitbreakpoint())

In [ ]:
memory.registerwrite(0x3F, 1, lambda addr,size: None)

In [ ]:
#print info on screen, when game write to selected address
memory.registerwrite(0x3F, 1, onExecute)

In [ ]:
#Read info about pressed buttons
#numeration of joypads starts from 1!
joypad.readimmediate(1)

In [ ]:
#Send joypad button keys
#All buttons must be filled #press START!
a = { "A" : True, "B":True, "down":False, "right":False, "select":False, "start":True, "up":False }
joypad.write(1, a)

In [ ]:
#Read mouse position
zapper.read()

In [ ]:
#Read keyboard info
input.read()

In [ ]:
#Check if button pressed every frame
def onEndFrame():
    keys = input.read()
    if keys.get("N"):
        emu.message("N pressed")

emu.registerafter(onEndFrame)

In [ ]:
#Draw rect on screen
def draw():
    gui.line(0,0,100,100, "red", False)
    gui.box(50,50,100,100, "FFFFFFFF", "white")
    gui.pixel(75,75,"blue")
    gui.text(80,80, "hello", "red", "white")
    
emu.registerafter(draw)

In [ ]:
#Check error in python #test callback with error
def error():
    1/0
emu.registerafter(error)

In [ ]:
emu.registerafter(None)

In [ ]:
#Now change rom
SUPER_MARIO = "Super Mario Bros. (JU) [!].nes"
emu.loadrom(SUPER_MARIO)

In [ ]:
def smbGetEnemyCoord(n):
    x2 = memory.readbyte(0x006D + n)
    x1 = memory.readbyte(0x0086 + n)
    y2 = memory.readbyte(0x00B5 + n)
    y1 = memory.readbyte(0x00CE + n)
    #ax = memory.readbytesigned(0x0057 + n)
    #ay = memory.readbytesigned(0x009F + n)
    ax = memory.readbytesigned(0x0057 + n)
    ay = memory.readbytesigned(0x009F + n)
    x = x2*256+x1
    y = y2*256+y1
    return x,y,ax,ay

In [ ]:
#Show info about first enemy slot on screen
def printEnemyInfo():
    info = smbGetEnemyCoord(1)
    emu.message("x:%d y:%d ax:%d ay:%d"%info)
    
emu.registerafter(printEnemyInfo)

In [ ]:
def smbMoveEnemy(n, x, y, ax, ay):
    x1 = x % 0x100
    x2 = x // 0x100
    y1 = y % 0x100
    y2 = y // 0x100
    ax = max(-128, min(ax, 0x7F))
    ay = max(-128, min(ay, 0x7F))

    memory.writebyte(0x006D + n, x2)
    memory.writebyte(0x0086 + n, x1)
    memory.writebyte(0x00B5 + n, y2)
    memory.writebyte(0x00CE + n, y1)
    memory.writebyte(0x0057 + n, ax)
    memory.writebyte(0x009F + n, ay)

    direction = ax > 0 and 1 or 2
    memory.writebyte(0x0045 + n, direction);

In [ ]:
#And now, try to change enemy's position and speed
smbMoveEnemy(1, 650, 440, 32, 0)

In [ ]:
smbMoveEnemy(1, 650, 430, 16, -8)

In [ ]:
emu.poweron()

In [ ]:
#Show position off scroll
def smbGetScreenPos():
    return memory.readbyte(0x71a) * 256 + memory.readbyte(0x71c)

In [ ]:
smbGetScreenPos()

In [ ]:
def showScreenPos():
    emu.message("Screen pos: %d"%smbGetScreenPos())
    
emu.registerafter(showScreenPos)

In [ ]:
#We can just move enemy with mouse clicks
def onMouseClick():
    inp = input.read()
    mouseX, mouseY = inp["xmouse"], inp["ymouse"]
    click = inp["click"]==1
    if click:
        x = smbGetScreenPos() + mouseX
        y = mouseY + 256
        smbMoveEnemy(1, x, y, 0, 0)
    
emu.registerafter(onMouseClick)

In [ ]:
#that's all!
#thank you for watching.